In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
# import constant data
# set concentration
concentration = np.array([0, 0.025, 0.0429, 0.0875, 0.1222, 0.15, 0.2, 0.2333, 0.275])

# set other constants
T = 30 + 273.15  # K
R = 8.314  # J/(mol*K)

gamma_water = 71.18  # mN/m

In [ ]:
# data_gen
raw_data = []

for i in range(0, 9):
  raw_data.append(np.loadtxt(f'data/surface_adsorption/0{i + 1}.txt', skiprows=1, delimiter=',').T[1])

data = []

# filter data
for _raw_data in raw_data:
  _temp = []
  _min = np.min(_raw_data)
  for i in range(0, len(_raw_data)):
    if (
      _raw_data[i] - _min <= 10
      and i != 0
      and i != len(_raw_data) - 1
      and _raw_data[i] < _raw_data[i - 1]
      and _raw_data[i] < _raw_data[i + 1]
    ):
      _temp.append(_raw_data[i])
  data.append(_temp)

for d in data:
  # inspect data length
  print(len(d), end=' ')

# use average as target value
avg = np.array([np.mean(d) for d in data])

with open('data/surface_adsorption/avg_minimal_pressure.csv', 'w') as f:
  f.write('concentration,minimal_pressure\n')  # header
  for i in range(0, len(concentration)):
    f.write(f'{concentration[i]:.4f},{avg[i]:.3f}\n')

# clac the machine constant by zero concentration
K = gamma_water / avg[0]

gamma = K * avg

gamma_ = gamma[1:]

In [ ]:
# gamma - c


In [ ]:
# gamma - ln(c)

# clac ln(concentration)
# ignore the first one (zero concentration)
ln_concentration = np.log(concentration[1:])

# linear regression
slope, intercept, r_value, p_value, std_err = st.linregress(ln_concentration, gamma_)

print(f'slope: {slope}')
print(f'intercept: {intercept}')
print(f'r2: {r_value**2}')

Gamma_inf = -slope / R / T * 1e-3
print(f'Gamma_inf: {Gamma_inf:e}')

_ln_c_new = np.linspace(ln_concentration[0], ln_concentration[-1], 100)
_gamma_new = [slope * x + intercept for x in _ln_c_new]

# plot gamma-ln(c)
plt.plot(ln_concentration, gamma_, 'o')
plt.plot(_ln_c_new, _gamma_new, '--')
plt.xlabel('ln(c)')
plt.ylabel('gamma')
plt.title('gamma-ln(c)')
plt.show()

# without last two data points
_slope, _intercept, _r_value, _p_value, _std_err = st.linregress(ln_concentration[:-2], gamma_[:-2])

print(f'slope: {_slope}')
print(f'intercept: {_intercept}')
print(f'r2: {_r_value**2}')

_Gamma_inf = -_slope / R / T * 1e-3
print(f'Gamma_inf: {_Gamma_inf:e}')

__ln_c_new = np.linspace(ln_concentration[0], ln_concentration[-1], 100)
_gamma_new = [_slope * x + _intercept for x in __ln_c_new]

# plot gamma-ln(c)
plt.plot(ln_concentration, gamma_, 'o')
plt.plot(__ln_c_new, _gamma_new, '--')
plt.xlabel('ln(c)')
plt.ylabel('gamma')
plt.title('gamma-ln(c) without last two data points')
plt.show()

In [ ]:
# plot gamma-c
# _c_new = [np.exp(x) for x in _ln_c_new]
# plt.plot(concentration[1:], gamma_, 'o')
# plt.plot(_c_new, _gamma_new, '--')
# plt.xlabel('c')
# plt.ylabel('gamma')
# plt.show()

In [ ]:
# def dif(c):
#   return slope / c

Z = [-slope / c for c in concentration[1:]]
Gamma = [z / R / T * 1e-3 for z in Z]

c_Gamma = []

for i in range(0, len(concentration[1:])):
  c_Gamma.append(concentration[1:][i] / Gamma[i])

slope_, intercept_, r_value_, p_value_, std_err_ = st.linregress(concentration[1:], c_Gamma)
print(f'slope: {slope_}')
print(f'intercept: {intercept_}')
print(f'r2: {r_value_**2}')

c_new_ = np.linspace(concentration[1], concentration[-1], 100)
c_Gamma_new_ = [slope_ * x + intercept_ for x in c_new_]

Gamma_inf_ = 1 / slope_
print(f'Gamma_inf: {Gamma_inf_:e}')

plt.plot(concentration[1:], c_Gamma, 'o')
plt.plot(c_new_, c_Gamma_new_, '--')
plt.xlabel('c')
plt.ylabel('c / Gamma')
plt.show()

In [ ]:
# claculate the error
Gamma_inf_err = (Gamma_inf - Gamma_inf_) / Gamma_inf
print(f'Gamma_inf_err: {Gamma_inf_err * 100}%')